# **ETL (Extract, Transform, Load)**

## Objectives
- Load the raw **VGChartz Video Game Sales** dataset and prepare it for analysis and dashboarding.  
- Perform basic data profiling to understand structure and quality.  
- Clean and transform the dataset (handle missing values, unify formats, engineer features such as multi-platform indicator, first-party flag, and release era).  
- Export a cleaned, analysis-ready dataset for use in visualizations and Tableau.

## Inputs
- **Raw data file:** `data/raw/Video_Games_Sales_as_at_22_Dec_2016.csv`  
- **Columns used:**  
  `Name`, `Platform`, `Year_of_Release`, `Genre`, `Publisher`,  
  `NA_Sales`, `EU_Sales`, `JP_Sales`, `Other_Sales`, `Global_Sales`,  
  `Critic_Score`, `Critic_Count`, `User_Score`, `User_Count`, `Developer`, `Rating`  
- **Python libraries:** `pandas`, `numpy`, `matplotlib`, `seaborn` (for quick profiling)

## Outputs
- **Processed dataset:** `data/processed/video_game_sales_clean.csv` — cleaned and feature-engineered for analysis.  
- Summary of data issues and cleaning actions in the ETL notebook (`notebooks/etl.ipynb`).  
- Basic exploratory statistics (row counts, missing values, data types) for reference.

## Additional Comments
- Major cleaning steps include:  
  - Removing rows with no game name or no sales data.  
  - Converting year to integer and handling missing or unrealistic years.  
  - Dropping or flagging games without review scores when needed for hypotheses.  
  - Creating new features:  
    - `Vendor` (Nintendo, Sony, Microsoft, Other)  
    - `is_multiplatform` (1 if game appears on ≥2 platforms)  
    - `is_first_party` (1 if publisher matches platform vendor)  
    - `Era` (pre-2010 vs post-2010 for trend analysis)  
- This notebook produces the single source of truth dataset used throughout the project (analysis, testing, and Tableau dashboard).


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

Set up the data directories

In [ ]:
# Set the file path for the raw data
raw_data_dir = os.path.join(current_dir, 'data/raw')

# Set the file path for the processed data
processed_data_dir = os.path.join(current_dir, 'data/processed')

In [ ]:
print("Raw data directory:", raw_data_dir)
print("Processed data directory:", processed_data_dir)

# Imports

Import the necessary packages to perform the ETL process.

In [ ]:
import numpy as np
import pandas as pd

from utils.cleaning import remove_review_cols, convert_dtypes, clean_data
from utils.game_merger import build_merged_df

# Load the data

In [ ]:
df = pd.read_csv(os.path.join(raw_data_dir, 'video_game_sales.csv'))

df.head()

# Data Profiling

Understanding the structure and basic info of the dataframe

In [ ]:
df.shape

This dataset contains 16719 rows and 16 columns

## Check and convert datatypes

In [ ]:
df.info()

### Data Type Adjustments

To prepare the dataset for analysis, several columns were converted to more suitable data types:

- **Year_of_Release** → changed from `float64` to `Int64` (nullable integer) to store whole years and handle missing values.
- **Critic_Score** → optionally converted to `Int64` since scores are whole numbers.
- **Platform, Genre, Publisher, Developer, Rating** → converted from `object` to `category` to reduce memory use and speed up grouping/filtering.

These changes make the dataset cleaner, improve performance, and prevent issues when running statistical tests or creating visualisations.


In [ ]:
df = convert_dtypes(df)

df.info()

In [ ]:
df.describe(include='all')

# Check for missing values

In [ ]:
df.isna().sum()

**Key observations:**

- **Sales data** (`NA_Sales`, `EU_Sales`, `JP_Sales`, `Other_Sales`, `Global_Sales`) is complete — no missing values.
- **Core identifiers** (`Name`, `Platform`, `Genre`, `Publisher`) are mostly complete, with only a few missing entries.
- **Year_of_Release** has 269 missing values — these may need to be dropped or imputed.
- **Review data** (`Critic_Score`, `Critic_Count`, `User_Score`, `User_Count`) is missing for about **50–55% of games**. This limits sample size for review-based hypotheses but is acceptable if we focus only on reviewed games for those analyses.
- **Developer and Rating** have ~40% missing — these are less critical but should be noted if we use them.
- `Name` and `Genre` each have only 2 missing entries — negligible and can be dropped.

**Implications for cleaning:**

- I will likely **drop rows with missing `Name` or `Global_Sales`** (key identifiers and target variable).
- For analyses involving reviews, we’ll use the subset with non-null `Critic_Score` or `User_Score`.
- I will consider dropping or flagging rows with missing `Year_of_Release` if time-based trends matter.
- Missing `Developer` and `Rating` can be ignored for now since they’re not central to chosen hypotheses.


# Check for duplicate values

In [ ]:
df.duplicated().sum()

As we can see above there are no exact duplicate rows.

Next I will check if there are any game titles that are duplicated.

In [ ]:
df[df.duplicated(subset=['Name'])].sort_values(by='Name')

As we can see here there are games that have duplicates.

This is because there are games with the same name released on different consoles.

These could be ports or remakes.

Later in this notebook I will convert these to a dataframe where they will be combined to give the total sales of a certain game across all platforms that they are released on.

# Data Cleaning

Remind ourselves of the null values

In [ ]:
# Check for missing values
df.isna().sum()

## Export 1 (Base Cleaned Dataset)

Before analysing the video game sales data, we need to create a cleaned and reliable version of the dataset.  
The main goals here are to ensure that key identifiers and target variables are present, handle missing values thoughtfully, and remove columns that are not needed for the first stage of analysis.

**Key cleaning steps:**
- **Remove incomplete rows** — Dropped any records missing a `Name` or `Global_Sales` value, since these are essential identifiers and target metrics.
- **Handle missing publishers and developers** — Replaced missing `Publisher` and `Developer` entries with `"Unknown"` to preserve the rows while marking incomplete data.
- **Drop unused columns** — Removed review-related fields (`Critic_Score`, `Critic_Count`, `User_Score`, `User_Count`) and `Rating` since this first export focuses on sales and platform data only.
- **Handle missing release years** — Replaced missing `Year_of_Release` with `-1` to keep the data but clearly mark unknown years. Converted the column to integer type for consistency.

The result is a **clean, analysis-ready dataset** that focuses purely on sales, platforms, publishers, developers, and release years.
    

In [ ]:
df_cleaned = df.copy()

df_cleaned = clean_data(df_cleaned)

df_cleaned.isna().sum()

In [ ]:
df_cleaned.info()

In [ ]:
df_cleaned.describe(include='all')

In [ ]:
# Save the cleaned data to a new CSV file
df_cleaned.to_csv(os.path.join(processed_data_dir, 'video_game_sales_cleaned.csv'), index=False)

# First Party Dataframe

Add `is_first_party` column to the cleaned dataframe

I have added it into a separate dataframe to the merged one in the next section as this one features the game sales per platform not as a whole.

This way I can look at if first party games generally sell better on their own platforms.

In [ ]:
# Get a list of unique platforms
platforms = df_cleaned['Platform'].unique().tolist() # Get unique platforms

platforms.sort() # Sort the list alphabetically

print(platforms)

In [ ]:
df_first_party = df_cleaned.copy()

# define platform families (manually curated)
nintendo = ['Wii', 'NES', 'SNES', 'GB', 'DS', '3DS', 'N64', 'GBA', 'WiiU', 'GC']
sony = ['PS', 'PS2', 'PS3', 'PS4', 'PS5', 'PSP', 'PSV']
microsoft = ['XB', 'X360', 'XOne', 'XSX']

# row-wise masks (Publisher vs Platform family)
mask_nin = (df_first_party['Publisher'].str.contains('nintendo', case=False, na=False)
            & df_first_party['Platform'].isin(nintendo))

mask_sony = (df_first_party['Publisher'].str.contains('sony|sce|sie|playstation', case=False, na=False)
             & df_first_party['Platform'].isin(sony))

mask_ms = (df_first_party['Publisher'].str.contains('microsoft', case=False, na=False)
           & df_first_party['Platform'].isin(microsoft))

# combine masks into one boolean column
df_first_party['is_first_party'] = (mask_nin | mask_sony | mask_ms)

df_first_party.head(25)


In [ ]:
df_first_party.to_csv(os.path.join(processed_data_dir, 'video_game_sales_first_party.csv'), index=False)
print("Merged dataframe saved to 'video_game_sales_first_party.csv'")

## Export 2 (Merged by Game Title)

After creating the base cleaned dataset where each game appears once per platform, the next step was to build a **title-level dataset**.  
This version combines all platform entries for the same game into a single row, making it easier to study overall game performance and trends without platform duplication.

**Key processing steps:**
- **Combine platform entries** — Grouped data by `Name` so each game title is represented once instead of one row per platform.
- **Sum sales metrics** — Added together `NA_Sales`, `EU_Sales`, `JP_Sales`, `Other_Sales`, and `Global_Sales` across all platforms to get total lifetime sales per game.
- **Aggregate platforms** — Created a `Platforms` column listing all unique platforms each game was released on.
- **Handle release year** — Selected the earliest known `Year_of_Release` for each game. If no year data was available, kept it as `Unknown`.
- **Simplify categorical data** — For fields like `Genre`, `Publisher`, and `Developer`, kept the most frequent value across platforms. If no single value dominated, marked it as `"Multiple"`.

The result is a **single-row-per-game dataset** that’s ideal for high-level analysis of game success, sales distribution, and market trends without platform-level duplication.


In [ ]:
df_merged = df.copy()

df_merged.head()

In [ ]:
df_merged = build_merged_df(df_merged,
                          max_year_span=5,
                          max_critic_diff=5.0,
                          require_same_publisher=False)

df_merged.head(20)

In [ ]:
df_merged = remove_review_cols(df_merged)

df_merged.head()

In [ ]:
# Add is_multiplatform column (True/False)
df_merged['is_multiplatform'] = df_merged['Platform'].str.contains(',').astype(bool)
df_merged.head()

In [ ]:
df_merged.to_csv(os.path.join(processed_data_dir, 'video_game_sales_merged.csv'), index=False)
print("Merged dataframe saved to 'video_game_sales_merged.csv'")

## Export 3 (Cleaned with Reviews Only)

The third dataset focuses specifically on **review-driven analysis**.  
Since critic and user scores are missing for about half the games in the full dataset, this export filters down to only games with available critic reviews, ensuring reliable insights when testing review-related hypotheses.

**Key processing steps:**
- **Filter for reviewed games** — Kept only rows where `Critic_Score` is available to maintain a consistent dataset for review-based analysis.
- **Retain review metrics** — Preserved `Critic_Score`, `Critic_Count`, `User_Score`, and `User_Count` so we can explore their relationships with global sales.
- **Keep core sales and metadata** — Retained important fields such as `Name`, `Platform`, `Year_of_Release`, `Genre`, `Publisher`, and `Developer` to allow segmentation by platform or genre while analyzing reviews.
- **Consistent cleaning rules** — Applied the same data cleaning steps as before (e.g., handling missing `Publisher`/`Developer` values, marking unknown years) to maintain data integrity across all exports.

The result is a **focused dataset for review and ratings analysis**, ideal for testing hypotheses like *“Do higher critic scores correlate with increased global sales?”* and creating visualizations that explore the impact of reviews on game success.


### Exported Datasets

For this project, I am creating and exporting three cleaned versions of the dataset.  
Each serves a slightly different analytical purpose and keeps the workflow flexible.

---

#### `video_game_sales_clean.csv` — **Cleaned (no critic or user scores)**
- Purpose: Base dataset for **general sales, platform, genre, and region analysis** where review data is not required.
- Changes made:
  - Removed rows with missing `Name` or `Global_Sales` (core identifiers and target variable).
  - Replaced missing `Publisher` and `Developer` values with `"Unknown"`.
  - Left missing `Year_of_Release` labeled as `"Unknown"` to keep data but mark uncertainty.
  - Dropped critic and user review columns (`Critic_Score`, `Critic_Count`, `User_Score`, `User_Count`) to simplify analysis and dashboard builds.

---

#### `video_game_sales_merged_by_title.csv` — **Cleaned with multiplatform emphasis**
- Purpose: Use for **platform-level and vendor comparisons** (e.g., Nintendo vs Sony vs Microsoft).
- Based on the same cleaned data as above but ensures each game has all its platforms associated with it.  
- Allows robust analysis of sales per game and region without review-related columns.

---

#### `video_game_sales_clean_reviews.csv` — **Cleaned with reviews only**
- Purpose: Specific dataset for testing **hypotheses around critic scores and user scores**.
- Created by filtering the data to include only games with a valid `Critic_Score`.
- Keeps review-related columns (`Critic_Score`, `Critic_Count`, `User_Score`, `User_Count`) for deeper statistical analysis.
- Useful for exploring relationships like “Do better critic scores drive higher sales?”

---

### Why this approach
By splitting the cleaned data into three purpose-built CSVs:
- Analysis and dashboards remain **lightweight** when reviews aren’t needed.
- Review-driven insights can still be explored without dealing with ~50% missing scores in the full dataset.
- Platform-level insights remain clean and reliable.

These files are saved in the `data/processed/` folder and serve as the single sources of truth for all subsequent analysis and visualisation.


---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)
